# Step 2 - Consume your data through the python connector and dump it in mongodb atlas.

In [1]:
import argparse

from confluent_kafka import Consumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.serialization import SerializationContext, MessageField
from confluent_kafka.schema_registry.json_schema import JSONDeserializer

In [3]:
# Kafka essentials details

# Variables
API_KEY = 'AMTQUJ4OYGGMNYOJ'
ENDPOINT_SCHEMA_URL  = 'https://psrc-8qyy0.eastus2.azure.confluent.cloud'
API_SECRET_KEY = '4HTAJgLsRtyeQjCpKZItrYiCs7eduapoIzAkFApIZer4nzPQ2i53tPWe58TGuPY/'
BOOTSTRAP_SERVER = 'pkc-n00kk.us-east-1.aws.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = 'BS3PHBXGYPRUDGFP'
SCHEMA_REGISTRY_API_SECRET = 'gMwNVdkw1Cn5pFHmtFKrZam64E5HeDEo1dJu2hxCcUSSjaIkuscBUw6ucAlmjT2l'

In [4]:
def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf

def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,
    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"
    }

class Car:   
    def __init__(self,record:dict):
        for k,v in record.items():
            setattr(self,k,v)
        
        self.record=record
   
    @staticmethod
    def dict_to_car(data:dict,ctx):
        return Car(record=data)

    def __str__(self):
        return f"{self.record}"


def kafka_consumer(topic, schema_id):
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    print('reading schema')
    schema_str = schema_registry_client.get_schema(schema_id).schema_str
    print(schema_str)
    json_deserializer = JSONDeserializer(schema_str,
                                         from_dict=Car.dict_to_car)

    print("deserializer start")
    consumer_conf = sasl_conf()
    consumer_conf.update({
                     'group.id': 'group1',
                     'auto.offset.reset': "latest"})

    print('consumer')
    consumer = Consumer(consumer_conf)
    consumer.subscribe([topic])


    while True:
        try:
            # SIGINT can't be handled when polling, limit timeout to 1 second.
            msg = consumer.poll(1.0)
            if msg is None:
                continue

            car = json_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))

            if car is not None:
                print("User record {}: car: {}\n"
                      .format(msg.key(), car))
        except KeyboardInterrupt:
            break

    consumer.close()

## Case CSV file

In [7]:
topic = 'topic_case'
schema_id = 100005

In [ ]:
kafka_consumer(topic, schema_id)

## Patientinfo CSV file

In [5]:
topic = 'topic_patientinfo'
schema_id = 100007

In [ ]:
kafka_consumer(topic, schema_id)

## Policy CSV file

In [10]:
topic = 'topic_policy'
schema_id = 100008

In [11]:
kafka_consumer(topic, schema_id)

reading schema
{"$id":"http://example.com/myURI.schema.json","$schema":"http://json-schema.org/draft-07/schema#","additionalProperties":false,"description":"Sample schema to help you get started.","title":"SampleRecord","type":"object","properties":{"policy_id":{"type":"integer"},"country":{"type":"string"},"type":{"type":"string"},"gov_policy":{"type":"string"},"detail":{"type":"string"},"start_date":{"type":"string"},"end_date":{"type":"string"}}}
deserializer start
consumer
User record b'94c363d3-3cb5-4e09-8cb7-e36e970b5ba0': car: {'policy_id': 2, 'country': 'Korea', 'type': 'Alert', 'gov_policy': 'Infectious Disease Alert Level', 'detail': 'Level 2 (Yellow)', 'start_date': '2020-01-20', 'end_date': '2020-01-27'}

User record b'c4bad866-64fe-41b6-929c-d63984bd4621': car: {'policy_id': 1, 'country': 'Korea', 'type': 'Alert', 'gov_policy': 'Infectious Disease Alert Level', 'detail': 'Level 1 (Blue)', 'start_date': '2020-01-03', 'end_date': '2020-01-19'}

User record b'2e60f7ae-2418-4b

## Region CSV file

In [12]:
topic = 'topic_region'
schema_id = 100009

In [13]:
kafka_consumer(topic, schema_id)

reading schema
{"$id":"http://example.com/myURI.schema.json","$schema":"http://json-schema.org/draft-07/schema#","additionalProperties":false,"description":"Sample schema to help you get started.","title":"SampleRecord","type":"object","properties":{"code":{"type":"integer"},"province":{"type":"string"},"city":{"type":"string"},"latitude":{"type":"number"},"longitude":{"type":"number"},"elementary_school_count":{"type":"integer"},"kindergarten_count":{"type":"integer"},"university_count":{"type":"integer"},"academy_ratio":{"type":"number"},"elderly_population_ratio":{"type":"number"},"elderly_alone_ratio":{"type":"number"},"nursing_home_count":{"type":"integer"}}}
deserializer start
consumer
User record b'60f202c0-4bb9-4d1e-bd2b-82f70d3df0d0': car: {'code': 10010, 'province': 'Seoul', 'city': 'Gangnam-gu', 'latitude': 37.518421, 'longitude': 127.047222, 'elementary_school_count': 33, 'kindergarten_count': 38, 'university_count': 0, 'academy_ratio': 4.18, 'elderly_population_ratio': 13.

## SearchTrend CSV file

In [14]:
topic = 'topic_searchtrend'
schema_id = 100010

In [15]:
kafka_consumer(topic, schema_id)

reading schema
{"$id":"http://example.com/myURI.schema.json","$schema":"http://json-schema.org/draft-07/schema#","additionalProperties":false,"description":"Sample schema to help you get started.","title":"SampleRecord","type":"object","properties":{"date":{"type":"string"},"cold":{"type":"number"},"flu":{"type":"number"},"pneumonia":{"type":"number"},"coronavirus":{"type":"number"}}}
deserializer start
consumer
User record b'e17f4646-8380-49fd-aa45-f97fbddac9b4': car: {'date': '2016-01-02', 'cold': 0.13372, 'flu': 0.17135, 'pneumonia': 0.20826, 'coronavirus': 0.0089}

User record b'7aeaea4e-bc94-467a-8374-a2558c0e3655': car: {'date': '2016-01-01', 'cold': 0.11663, 'flu': 0.0559, 'pneumonia': 0.15726, 'coronavirus': 0.00736}

User record b'0b460031-1867-43dc-a846-d027a97aa17e': car: {'date': '2016-01-03', 'cold': 0.14917, 'flu': 0.22317, 'pneumonia': 0.19326, 'coronavirus': 0.00845}

User record b'b923ca18-6a02-4438-9f7f-3101ed8bbdcd': car: {'date': '2016-01-04', 'cold': 0.17463, 'flu'